<a href="https://colab.research.google.com/github/SecretPasta/Llama_Cloud/blob/main/Llama_Cloud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PIP Installs

In [15]:
!pip install firebase

# Importing Modules

In [16]:
import requests
from bs4 import BeautifulSoup
import json
from firebase import firebase
from IPython.display import HTML, IFrame, display, Javascript, clear_output
from google.colab import output


# Config

In [17]:
database = "https://llama-cloud-default-rtdb.europe-west1.firebasedatabase.app/"
# Define the base URL and search words
base_url = 'https://aws.amazon.com/'
search_words = ['AWS', 'Cloud Computing', 'EC2', 'S3', 'Lambda', 'IAM', 'VPC', 'RDS', 'Route', 'CloudFormation']
max_urls = 100

# Scrapper Code

Function to run web scrapper and save results in DB and JSON

In [30]:
# import requests
# from bs4 import BeautifulSoup
# import json
# from firebase import firebase

def scrape_and_save():
    # Initialize a dictionary to hold the results
    results={}
    results['term'] = {word: [] for word in search_words}

    # Keep track of URLs to visit, those already visited, and the link count for ranking
    urls_to_visit = [base_url]
    visited_urls = set()
    url_link_count = {}  # For page ranking

    while urls_to_visit:
        url = urls_to_visit.pop(0)
        visited_urls.add(url)

        # Send a GET request to the website
        response = requests.get(url)

        if response.status_code == 200:
            # Use BeautifulSoup to parse the HTML content
            soup = BeautifulSoup(response.text, 'html.parser')

            # Find all links in the webpage
            for link in soup.find_all('a', href=True):
                href = link['href']
                link_text = link.text.strip()
                full_url = requests.compat.urljoin(url, href)

                # Update page rank (link count)
                if full_url in url_link_count:
                    url_link_count[full_url] += 1
                else:
                    url_link_count[full_url] = 1

                # Check if the URL belongs to the base domain or its subdomains and hasn't been visited yet
                if 'aws.amazon.com' in full_url and full_url not in visited_urls:
                    if any(word.lower() in full_url.lower() for word in search_words):
                        urls_to_visit.append(full_url)

                    # Check if the link text or href contains any of the search words
                    for word in search_words:
                        if word.lower() in link_text.lower() or word.lower() in href.lower():
                            # Add the link to the results under the relevant search word
                            if full_url not in results['term'][word]:
                                results['term'][word].append({'DocIDs': full_url, 'rank': url_link_count[full_url]})

        # Limit to prevent excessive recursion or too many requests; adjust as needed
        if len(visited_urls) >= max_urls:
            break

    # Sort results by page rank before saving
    for word in results['term']:
        results['term'][word].sort(key=lambda x: x['rank'], reverse=True)

    # Save the results to a JSON file
    with open('aws_links_ranked.json', 'w') as json_file:
        json.dump(results, json_file, indent=4)

    print("Scraping completed and results saved to aws_links_ranked.json.")

    # Saving to DB
    FBconn = firebase.FirebaseApplication(database, None)  # Ensure you replace 'your_firebase_database_url' with your actual Firebase database URL.
    result = FBconn.post('/index/', results)
    print(result)

scrape_and_save()

Scraping completed and results saved to aws_links_ranked.json.
{'name': '-Ns-EG1h3chCy_PsoZaN'}


# Landing Page HTML

In [19]:
landing_page_html = """
<!DOCTYPE html>
<html lang="en">

<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Llama</title>

    <!-- For PNG or other formats -->
    <link rel="icon" type="image/png" href="logo_nobg.png">
    <!-- Apple Touch Icon for use with Apple devices -->
    <link rel="apple-touch-icon" href="logo_nobg.png">
    <!-- To specify a theme color for address bar in mobile browsers -->
    <meta name="theme-color" content="#10182F">

    <!-- Function to change the title of the page when the tab is not active -->
    <script>
        function handleVisibilityChange() {
            if (document.hidden) {
                // Set the new title when the tab is inactive
                document.title = "Llama 😢";
            } else {
                // Reset the title when the tab is active again
                document.title = "Llama";
            }
        }
        // Add a listener for the visibilitychange event
        document.addEventListener("visibilitychange", handleVisibilityChange, false);
    </script>

    <link href="https://fonts.googleapis.com/css2?family=Roboto:wght@400;700&display=swap" rel="stylesheet">
    <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/5.15.3/css/all.min.css">
    <style>
        * {
            box-sizing: border-box;
            margin: 0;
            padding: 0;
        }

        body,
        html {

            -webkit-user-select: none;
            /* Safari */
            -moz-user-select: none;
            /* Firefox */
            -ms-user-select: none;
            /* Internet Explorer/Edge */
            user-select: none;
            /* Non-prefixed version, currently supported by Chrome, Opera and Edge */

            height: 100%;
            font-family: 'Roboto', sans-serif;
            background: linear-gradient(to bottom left, #0F172A, #1A2346, #193251, #142941, #0F172A, #0F172A, #0F172A);
            /* Dark background */
            background-attachment: fixed;
            /* Keep the gradient fixed during scroll */
            color: white;
        }

        /* --- HEADER --- */
        header {
            position: absolute;
            display: flex;
            top: 0;
            width: 100%;
            padding: 1rem;
            justify-content: space-between;
            align-items: center;
        }

        header .login,
        header .user-icon {
            color: #94A3B8;
        }

        header .user-icon {
            font-size: 2rem;
        }

        .logo-container {
            top: 0;
            right: 0;
            padding: 10px;
            /* Adjust the padding to control the distance from the corner */
            display: flex;
            /* Enables Flexbox for this container */
            align-items: center;
            /* Vertically centers the flex items (the image and label) */
            justify-content: flex-start;
            /* Aligns items to the start of the container */
        }

        .logo {
            height: 100px;
            /* Adjust based on your logo's size */
            width: auto;
            /* Maintains the aspect ratio of the logo */
            margin-right: 10px;
            /* Adds some space between the logo and the label */
        }

        .miniTitle {
            font-size: 1.5rem;
            font-weight: 600;
            color: #38BDF8;
        }

        .title {
            font-size: 6rem;
            font-weight: 600;
            background: linear-gradient(to right, #007bff, #70bee6);
            /* Gradient from blue to light blue */
            -webkit-background-clip: text;
            -webkit-text-fill-color: transparent;
            display: inline;
            /* Required for gradient text */
            background-clip: text;
            color: transparent;
            /* Fallback for non-webkit browsers */
            text-shadow: 0 0 3px #007bff59, 0 0 6px #007bff56, 0 0 9px #85d6ff50;
            /* Glowing effect */
            font-weight: bold;
        }

        .login-container {
            text-decoration: none;
            /* Remove underline from links */
            display: flex;
            gap: 6px;
            align-items: center;
            justify-content: center;
            color: #94A3B8;
            margin: 20px;
            background-color: #ffffff0f;
            border: 1px solid #ffffff1a;
            width: auto;
            min-width: 120px;
            border-radius: 10px;
            padding: 10px;
            backdrop-filter: blur(10px);
            position: fixed;
            right: 0;
            transition: background-color 0.3s ease;
            cursor: pointer;
            /* Make the cursor indicate clickable area */
        }

        .login-container:hover {
            background-color: #ffffff1a;
        }

        /* --- MAIN --- */

        main {
            display: flex;
            justify-content: center;
            align-items: center;
            min-height: 100vh;
            /* Use at least 100% of the viewport height */
            width: 100vw;
            /* Use 100% of the viewport width */
        }

        main .container {
            display: flex;
            flex-direction: column;
            justify-content: center;
            align-items: center;
            text-align: center;
            /* Change height to 100vh to fill the entire viewport */
        }

        .welcome-message {
            display: flex;
            flex-direction: column;
            align-items: center;
            justify-content: center;
            height: 30vh;
            /* This takes up the full height of the viewport */
            text-align: center;
            margin-bottom: 30px;
        }


        .welcome-message p {
            font-size: 1.25rem;
            /* Larger font size for the subtitles */
            margin-bottom: 1rem;
            /* Spacing between paragraphs */
            margin-top: 1rem;
        }

        .welcome-message .cta-text {
            font-size: 1.5rem;
            /* Even larger font size for call-to-action */
            font-weight: bold;
            /* Make it bold */
            color: #ff9900;
            /* Make it a color that stands out */
            animation: pulse 2s infinite;
            /* Add an animation to attract attention */
        }

        /* Keyframes for the pulse animation */
        @keyframes pulse {
            0% {
                transform: scale(1);
            }

            50% {
                transform: scale(1.05);
            }

            100% {
                transform: scale(1);
            }
        }


        .search-box {
            display: flex;
            /* This will align the children (input and button) in a row */
            align-items: center;
            /* This will vertically center the input and button */
            overflow: hidden;
            /* Clip the children to the wrapper's border radius */
            border-radius: 50px;
            /* Your desired border radius */

            border: 2px solid #0488D2;
            /* New: Unified border style */
            background: #ffffff1a;
            /* New: Unified background */
            /* box-shadow: 0 2px 4px #007bff88; */
            /* New: Default shadow, consider conditional application */

        }

        .search-box:focus-within {
            box-shadow: 0 0 8px #007bff;
            /* New box-shadow when input is focused */
            border-color: #007bff;
            /* Change border color on hover */
        }

        /* Small x in the search bar right corner */
        .search-box input[type="search"]::-webkit-search-cancel-button {
            -webkit-appearance: none;
            height: 0;
            width: 0;
            display: none;
        }

        .search-box:hover {
            border-color: #007bff;
            /* Change border color on hover */
            box-shadow: 0 0 8px #007bff88;
            /* Subtle shadow for depth */
        }

        .search-box input[type="search"] {
            color: #E2E8F0;
            padding: 1rem 0.5rem;
            font-size: 1rem;
            width: 30rem;
            font-weight: bold;
            border: none;
            /* Adjusted: Removed border */
            background: transparent;
            /* Adjusted: Background transparent */
        }

        .search-box input[type="search"]::placeholder {
            color: #94A3B8;
            font-weight: bold;
        }

        /* When pressed on search */
        .search-box input[type="search"]:focus {
            outline: none;
        }

        .search-box .search-btn,
        .left-pan {
            padding: 13px 20px;
            font-size: 1.3em;
            border: none;
            /* Adjusted: Removed individual borders */
            background: transparent;
            /* Adjusted: Background transparent */
            color: #94A3B8;
            cursor: pointer;
        }

        .search-box .search-btn i,
        .left-pan .divider {
            pointer-events: none;
            /* Prevents the icon from being clickable separately from the button */
        }

        .search-box .search-btn:hover {
            background: linear-gradient(to bottom left, #0F172A, #10182F);
            /* border-color: #0d6efd; */
            color: #E2E8F0;
        }

        .left-pan .divider {
            color: #94A3B8;
            /* Light grey color for the divider */
            font-size: 1.4rem;
            font-weight: bold;
        }

        .left-pan:hover {
            color: #E2E8F0;
            /* Change background color on hover */
            cursor: pointer;
        }

        .left-pan i.fa-microphone {
            /* Apply glow effect to the microphone icon */
            text-shadow: 0 0 2px #007bff;
        }

        .search-btn i.fa-search {
            /* Apply glow effect to the search icon */
            text-shadow: 0 0 3px #0F172A;
        }

        /* Optional: Enhance the glow on hover */
        .left-pan:hover i.fa-microphone {
            text-shadow: 0 0 12px #007bff;
            /* Stronger glow effect on hover */
        }


        .filter-options {
            display: flex;
            justify-content: space-evenly;
            align-items: center;
            /* This will vertically center the items in the container */
            gap: 2rem;
            /* Space between filter buttons */
            margin: 1.3rem 0;
            /* Distance from the search box */
            flex-wrap: wrap;
            /* Ensure responsiveness */
        }

        .filter-topic,
        .filter-btn {
            padding: 10px 15px;
            border: 2px solid #0488D2;
            /* Match the search box border */
            background-color: #ffffff1a;
            /* Semi-transparent background like the search box */
            color: #94A3B8;
            /* Light grey color to match the icon and placeholder */
            font-weight: bold;
            /* For emphasis */
            border-radius: 50px;
            /* Rounded corners */
            cursor: pointer;
            transition: background-color 0.3s, color 0.3s;
            /* Smooth transition for hover effects */
        }

        .filter-btn {
            background: linear-gradient(to bottom left, #0F172A, #1A2346);
            /* Gradient background for filter button */
            color: #E2E8F0;
            /* White-ish color for text to stand out */
        }

        .filter-topic:hover,
        .filter-btn:hover {
            background-color: #0488D2;
            /* Darker color on hover for visibility */
            color: #FFFFFF;
            /* White color for text on hover */
            border-color: #007bff;
            /* Slightly lighter border on hover */
        }

        .filter-topic.pressed {
            background-color: #0056b3;
            /* Darker shade to indicate pressed state */
            color: #FFFFFF;
            /* Change text color for contrast */
            border-color: #004080;
            /* Optionally change the border color */
            box-shadow: 0 0 4px #007bff88;
            /* Subtle shadow for depth */
        }


        .more-filters {
            display: flex;
            flex-direction: column;
            align-items: center;
            margin-top: 20px;
            padding: 10px;
            background-color: #0F172A;
            /* Deep background to contrast with button */
            border-radius: 10px;
            /* Rounded corners for the container */
            box-shadow: 0 4px 8px rgba(0, 0, 0, 0.2);
            /* Soft shadow for depth */
        }

        .checkbox-container {
            margin: 10px 0;
        }

        .checkbox-container input[type="checkbox"] {
            /* Hide the default checkbox */
            -webkit-appearance: none;
            -moz-appearance: none;
            appearance: none;
            /* Create a custom checkbox appearance */
            width: 20px;
            height: 20px;
            background-color: #ffffff1a;
            border: 2px solid #0488D2;
            border-radius: 4px;
            cursor: pointer;
            position: relative;
        }

        .checkbox-container input[type="checkbox"]:checked {
            background-color: #007bff;
            border: 2px solid #007bff;
        }

        .checkbox-container input[type="checkbox"]:checked::after {
            /* Custom checkmark style */
            content: '';
            position: absolute;
            left: 5px;
            top: 1px;
            width: 6px;
            height: 12px;
            border: solid white;
            border-width: 0 3px 3px 0;
            transform: rotate(45deg);
        }

        .checkbox-container label {
            margin-left: 8px;
            color: #94A3B8;
            /* Light grey color to match the design */
            font-weight: bold;
            user-select: none;
            /* Prevent text selection */
        }

        /* --- FOOTER --- */

        footer {
            position: absolute;
            bottom: 0;
            width: 100%;
            padding: 1rem;
            text-align: center;
            background: transparent;
        }

        footer a {
            color: #E2E8F0;
            /* Light grey color for footer links */
            text-decoration: none;
            font-size: 1rem;
            margin: 0 1rem;
            transition: color 0.3s ease;
        }

        footer a:hover {
            color: #0E99DA;
        }

        /* --- FOR DIFFERENT SIZES OF SCREEN --- */
        @media (max-width: 768px) {
            .title {
                font-size: 4rem;
                /* Smaller font size for smaller screens */
            }

            .miniTitle,
            .welcome-message p,
            .welcome-message .cta-text {
                font-size: 1rem;
                /* Adjust font size for readability */
            }

            .search-box input[type="search"] {
                font-size: 0.8rem;
                /* Smaller font size for the search input */
                width: 15rem;
                /* Adjust width to fit smaller screens */
            }

            header,
            .login-container,
            .welcome-message,
            main .container {
                padding: 0.5rem;
                /* Reduce padding */
                margin: 0.5rem;
                /* Adjust margins */
            }

            .logo {
                height: auto;
                /* Maintain aspect ratio */
                max-width: 15%;
                /* Ensure logo does not exceed container width */
            }

            .filter-options,
            .more-filters {
                display: none;
                /* Optionally hide filters on smaller screens */
            }
        }
    </style>
</head>

<body>

    <header>

        <div class="logo-container">
            <img src="logo_nobg.png" alt="LlamaCloud" class="logo">
            <!-- <span class="miniTitle">LlamaCloud</span> -->
        </div>

        <a onclick="openAdmin()" class="login-container">Login
            <i class="fas fa-user-circle user-icon"></i>
        </a>


    </header>

    <main>
        <div class="container">

            <!-- WELCOME MESSAGE -->
            <div class="welcome-message">
                <h1 class="title">
                    Llama
                </h1>
                <p>The best place to search AWS Services, Documentation, Blog Posts, and More...</p>
                <p class="cta-text">Discover AWS with a simple search<br>👇🏼</p>
            </div>

            <!-- SEARCH BOX -->
            <div class="search-box">
                <button class="left-pan" aria-label="Search by voice">
                    <i class="fa fa-microphone"></i>
                    <span class="divider">|</span>
                </button>

                <input id="search" type="search" placeholder="What are you looking for in AWS?">


                    <button onclick="performSearch()" class="search-btn" type="submit" aria-label="Search">
                        <i class="fas fa-search"></i>
                    </button>

            </div>

            <!-- FILTER OPTIONS -->
            <div class="filter-options">
                <button class="filter-topic">Server</button>
                <button class="filter-topic">Documents</button>
                <button class="filter-topic">Tools</button>
                <button class="filter-topic">APIs</button>
                <button class="filter-btn">Filters</button>
            </div>

            <!-- MORE FILTERS (Initially hidden)-->
            <div class="more-filters" style="display: none;">
                <div class="checkbox-container">
                    <input type="checkbox" id="option1" name="filter" value="option1">
                    <label for="option1">Option 1</label>
                </div>
                <div class="checkbox-container">
                    <input type="checkbox" id="option2" name="filter" value="option2">
                    <label for="option2">Option 2</label>
                </div>
                <div class="checkbox-container">
                    <input type="checkbox" id="option3" name="filter" value="option3">
                    <label for="option3">Option 3</label>
                </div>
            </div>


        </div>
    </main>

    <footer>
        <a href="https://youtu.be/dQw4w9WgXcQ" target="_blank">made with 💙 in Braude</a>
        <a href="https://github.com/SecretPasta/Llama_Cloud.git" target="_blank">GitHub</a>
    </footer>

    <!-- SCRIPT SECTION -->


    <!-- SEARCH QUERY + AUDIO INPUT -->
    <script>
        document.querySelector('.search-btn').addEventListener('click', function () {
            var query = document.getElementById('search').value;
            console.log("Search query:", query);
            // Implement search functionality here
        });

        // Example function to start audio input - this will need actual implementation
        document.querySelector('.left-pan').addEventListener('click', function () {
            console.log("Audio input activated");
            // Implement audio input functionality here
        });
    </script>

    <!-- BUTTONS STAY PRESSED AFTER CLICK -->
    <script>
        // Wait for the DOM to fully load
        document.addEventListener('DOMContentLoaded', function () {
            // Get all the filter-topic buttons
            var filterButtons = document.querySelectorAll('.filter-topic');

            // Add a click event listener to each button
            filterButtons.forEach(function (button) {
                button.addEventListener('click', function () {
                    // Toggle the 'pressed' class on click
                    button.classList.toggle('pressed');
                });
            });
        });
    </script>

    <!-- OPEN FILTERS ON CLICK -->
    <script>
        document.addEventListener('DOMContentLoaded', function () {
            const filterBtn = document.querySelector('.filter-btn');
            const moreFilters = document.querySelector('.more-filters');

            filterBtn.addEventListener('click', function () {
                // Toggle visibility
                if (moreFilters.style.display === 'none') {
                    moreFilters.style.display = 'block';
                } else {
                    moreFilters.style.display = 'none';
                }
            });
        });
    </script>
    <script>
      function performSearch() {
    // This will call the Python function 'display_results'
    google.colab.kernel.invokeFunction('notebook.display_results', [], {})
        .then(function(response) {
            // Handle response here if needed
        })
        .catch(function(error) {
            console.error(error);
        });
}
</script>

<script>
      function openAdmin() {
    // This will call the Python function 'display_admin'
    google.colab.kernel.invokeFunction('notebook.display_admin', [], {})
        .then(function(response) {
            // Handle response here if needed
        })
        .catch(function(error) {
            console.error(error);
        });
}
</script>


</body>

</html>


"""

# Search Results HTML

In [20]:
search_results = """
<!DOCTYPE html>
<html lang="en">

<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Llama Search Results</title>

    <!-- For PNG or other formats -->
    <link rel="icon" type="image/png" href="logo_nobg.png">
    <!-- Apple Touch Icon for use with Apple devices -->
    <link rel="apple-touch-icon" href="logo_nobg.png">
    <!-- To specify a theme color for address bar in mobile browsers -->
    <meta name="theme-color" content="#10182F">

    <link href="https://fonts.googleapis.com/css2?family=Roboto:wght@400;700&display=swap" rel="stylesheet">
    <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/5.15.3/css/all.min.css">
    <style>
        * {
            box-sizing: border-box;
            margin: 0;
            padding: 0;
        }

        body,
        html {
            -webkit-user-select: none;
            /* Safari */
            -moz-user-select: none;
            /* Firefox */
            -ms-user-select: none;
            /* Internet Explorer/Edge */
            user-select: none;
            /* Non-prefixed version, currently supported by Chrome, Opera and Edge */
            height: 100%;
            font-family: 'Roboto', sans-serif;
            background: linear-gradient(to bottom left, #0F172A, #1A2346, #193251, #142941, #0F172A, #0F172A, #0F172A);
            /* Dark background */
            color: white;
        }

        /* Header Styles */
        header {
            display: flex;
            flex-direction: column;
            /* Stack elements vertically */
            align-items: center;
            width: 100%;
            padding: 1rem;
        }

        .header-top {
            display: flex;
            width: 100%;
            justify-content: space-between;
            /* Space out logo, search bar, and login button */
            align-items: center;
        }

        /* Search Box Styles */
        .search-box {
            display: flex;
            align-items: center;
            overflow: hidden;
            border-radius: 50px;
            border: 2px solid #0488D2;
            background: #ffffff1a;
            flex-grow: 2;
            /* Allow search box to take up available space */
            margin: 0 10px;
            /* Ensure some space around the search box */
        }

        .search-box input[type="search"] {
            flex-grow: 1;
            padding: 10px;
            border: none;
            background-color: transparent;
            color: #94A3B8;
            font-size: 1rem;
            font-weight: bold;
        }

        .search-box .search-btn,
        .left-pan {
            background: none;
            border: none;
            padding: 10px;
            cursor: pointer;
            color: #94A3B8;
            /* Button icon color */
        }

        .search-box button {
            background: none;
            border: none;
            color: #94A3B8;
            cursor: pointer;
        }

        /* Filter Options Styles */
        .filter-options {
            display: flex;
            justify-content: center;
            /* Center filter buttons under the search bar */
            gap: 10px;
            width: 100%;
            margin-top: 10px;
            /* Space between search bar and filter buttons */
        }

        .filter-topic,
        .filter-btn {
            background: #ffffff1a;
            /* Semi-transparent background */
            border: 1px solid #0488D2;
            /* Border color */
            border-radius: 20px;
            /* Rounded edges */
            padding: 5px 15px;
            /* Padding inside the buttons */
            cursor: pointer;
            color: #E2E8F0;
            /* Button text color */
        }

        .filter-options button {
            background-color: transparent;
            border: 2px solid #0488D2;
            border-radius: 20px;
            padding: 5px 10px;
            color: #94A3B8;
            cursor: pointer;
            transition: background-color 0.3s ease, color 0.3s ease;
        }

        .filter-topic:hover,
        .filter-btn:hover {
            background: #0488D2;
            /* Darker background on hover */
            color: #ffffff;
            /* White text on hover */
        }

        header .login,
        header .user-icon {
            color: #94A3B8;
        }

        header .user-icon {
            font-size: 2rem;
        }

        .logo-container {
            top: 0;
            right: 0;
            padding: 10px;
            /* Adjust the padding to control the distance from the corner */
            display: flex;
            /* Enables Flexbox for this container */
            align-items: center;
            /* Vertically centers the flex items (the image and label) */
            justify-content: flex-start;
            /* Aligns items to the start of the container */
        }

        .logo {
            height: 80px;
            /* Adjust based on your logo's size */
            width: auto;
            /* Maintains the aspect ratio of the logo */
            margin-right: 10px;
            /* Adds some space between the logo and the label */
        }

        .miniTitle {
            font-size: 1.5rem;
            font-weight: 600;
            color: #38BDF8;
        }

        .title {
            font-size: 6rem;
            font-weight: 600;
            background: linear-gradient(to right, #007bff, #70bee6);
            /* Gradient from blue to light blue */
            -webkit-background-clip: text;
            -webkit-text-fill-color: transparent;
            display: inline;
            /* Required for gradient text */
            background-clip: text;
            color: transparent;
            /* Fallback for non-webkit browsers */
            text-shadow: 0 0 3px #007bff59, 0 0 6px #007bff56, 0 0 9px #85d6ff50;
            /* Glowing effect */
            font-weight: bold;
        }

        .login-container {
            display: flex;
            align-items: center;
            justify-content: center;
            padding: 8px;
            background-color: #ffffff0f;
            border: 1px solid #ffffff1a;
            border-radius: 50px;
            text-decoration: none;
            color: #94A3B8;
            transition: background-color 0.3s ease;
            cursor: pointer;
            font-size: 0.9rem;
            /* Adjust the font size if necessary */
        }

        .login-container i {
            margin-left: 5px;
            /* Space between text and icon */
        }

        .login-container:hover {
            background-color: #ffffff1a;
        }


        /* --- MAIN --- */

        .main {
            width: 100%;
            /* Ensure main content takes full width */
            padding-top: 1rem;
            /* Minimal space between header and main content */
            display: flex;
            justify-content: flex-start;
            align-items: center;
            flex-direction: column;
        }

        .search-container {
            display: flex;
            flex-direction: column;
            align-items: center;
            justify-content: center;
        }

        .search-results {
            background: #ffffff1a;
            /* Light background for sections */
            border-radius: 10px;
            padding: 20px;
            margin: 20px 0;
            width: 80%;
            /* Adjusted width for better layout control */
            max-width: 960px;
            /* Max width for larger screens */
            box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);
            /* Subtle shadow for depth */
        }

        .results-title {
            margin-bottom: 1rem;
            color: #38BDF8;
            /* Example color */
            text-align: center;
        }

        .search-result-item {
            padding: 1rem;
            margin-bottom: 1rem;
            border-bottom: #007bff 1px dashed;
            border-radius: 5px;
        }

        .result-link {
            color: #007bff;
            /* Example link color */
            font-weight: bold;
            text-decoration: none;
            /* Removes underline */
        }

        .result-description {
            color: #E2E8F0;
            /* Light grey for readability */
            padding-top: 0.5rem;
        }

        .pagination {
            display: flex;
            gap: 10px;
        }

        .pagination a {
            color: #94A3B8;
            text-decoration: none;
            padding: 5px 10px;
            border: 1px solid #94A3B8;
            border-radius: 5px;
            transition: background-color 0.3s, color 0.3s;
        }

        .pagination a:hover {
            background-color: #94A3B8;
            color: #ffffff;
        }

        .page-link,
        .pagination-ellipsis {
            color: #94A3B8;
            /* Styling for pagination links */
            text-decoration: none;
        }

        .page-link:hover {
            text-decoration: underline;
            /* Adds interactivity on hover */
        }




        /* --- FOOTER --- */

        footer {
            position: absolute;
            bottom: 0;
            width: 100%;
            padding: 1rem;
            text-align: center;
            background: transparent;
        }

        footer a {
            color: #E2E8F0;
            /* Light grey color for footer links */
            text-decoration: none;
            font-size: 1rem;
            margin: 0 1rem;
            transition: color 0.3s ease;
        }

        footer a:hover {
            color: #0E99DA;
        }
    </style>
</head>

<body>
    <header>
        <div class="header-top">
            <div class="logo-container">
                <img onclick = "openMainPage()" src="logo_nobg.png" alt="LlamaCloud" class="logo">
            </div>
            <div class="search-box">
                <button class="left-pan" aria-label="Search by voice">
                    <i class="fa fa-microphone"></i>
                    <span class="divider">|</span>
                </button>
                <input id="search" type="search" placeholder="What are you looking for in AWS?">
                <button class="search-btn" type="submit" aria-label="Search">
                    <i class="fas fa-search"></i>
                </button>
            </div>
            <a href="index.html" class="login-container">Log Out
                <i class="fas fa-sign-out-alt"></i>
            </a>
        </div>
        <div class="filter-options">
            <button class="filter-topic">Server</button>
            <button class="filter-topic">Documents</button>
            <button class="filter-topic">Tools</button>
            <button class="filter-topic">APIs</button>
            <button class="filter-btn">Filters</button>
        </div>
    </header>

    <main>
        <div class="search-container">

            <!-- Search Result -->
            <div class="search-result">
                <!-- Search Result Item -->
                <div class="search-result-item">
                    <a href="https://aws.amazon.com/account/sign-up" class="result-link">Amazon Web Services</a>
                    <p class="result-description">Get Innovating On AWS Today — Build, Deploy, and Manage Websites, Apps
                        or Processes On
                        AWS' Secure, Reliable Network. Sign Up for a Free Account & Experience AWS' Secure, Reliable,
                        Scalable Services.
                        In-Memory Caching. Performance At Scale. Easily Manage Clusters.</p>
                </div>
                <!-- Additional results here, structured similarly -->
                <div class="search-result-item">
                    <a href="#" class="result-link">What is Cloud Computing?</a>
                    <p class="result-description">Benefits & Types of Cloud Computing Learn How AWS Can Help your
                        Success</p>
                </div>
                <div class="search-result-item">
                    <a href="#" class="result-link">AWS Cloud Compute Service</a>
                    <p class="result-description">Lightsail: Easy-To-Use Platform For Simple Computing. 1 Year Free.</p>
                </div>

                <div class="search-result-item">
                    <a href="#" class="result-link">Why Choose AWS?</a>
                    <p class="result-description">From Startups to Enterprises. Learn Why Millions of Customers Chose
                        AWS</p>
                </div>
                <div class="search-result-item">
                    <a href="#" class="result-link">AWS Compute Solutions</a>
                    <p class="result-description">Try Amazon EC2 - Resizable Compute Capacity in The Cloud. 1 Year Free.
                    </p>
                </div>
                <div class="search-result-item">
                    <a href="#" class="result-link">Why Amazon Web Services</a>
                    <p class="result-description">Focus on What Differentiates Your Business - Not the Infrastructure.
                    </p>
                </div>
            </div>

            <!-- Pagination -->
            <div class="pagination">
                <a href="#" class="page-link">1</a>
                <a href="#" class="page-link">2</a>
                <a href="#" class="page-link">3</a>
                <span class="pagination-ellipsis">. . .</span>
                <a href="#" class="page-link">10</a>
            </div>
        </div>
    </main>

    <footer>
        <a href="https://youtu.be/dQw4w9WgXcQ" target="_blank">made with 💙 in Braude</a>
        <a href="https://github.com/SecretPasta/Llama_Cloud.git" target="_blank">GitHub</a>
    </footer>

<script>
      function openMainPage() {
    // This will call the Python function 'display_homepage'
    google.colab.kernel.invokeFunction('notebook.display_homepage', [], {})
        .then(function(response) {
            // Handle response here if needed
        })
        .catch(function(error) {
            console.error(error);
        });
}
</script>

</body>

</html>
"""

# Admin Section HTML

In [21]:
admin_page = """
<!DOCTYPE html>
<html lang="en">

<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Llama - Admin Dashboard</title>

    <!-- Icons -->
    <link rel="icon" type="image/png" href="logo_nobg.png">
    <link rel="apple-touch-icon" href="logo_nobg.png">
    <meta name="theme-color" content="#10182F">

    <!-- Fonts and Icons -->
    <link href="https://fonts.googleapis.com/css2?family=Roboto:wght@400;700&display=swap" rel="stylesheet">
    <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/5.15.3/css/all.min.css">

    <!-- Visibility Change Script -->
    <script>
        function handleVisibilityChange() {
            if (document.hidden) {
                document.title = "Llama 😢";
            } else {
                document.title = "Llama";
            }
        }
        document.addEventListener("visibilitychange", handleVisibilityChange, false);
    </script>

    <style>
        body,
        html {
            margin: 0;
            padding: 0;
            overflow-x: hidden;
            font-family: 'Roboto', sans-serif;
            background: linear-gradient(to bottom left, #0F172A, #1A2346, #193251, #142941, #0F172A);
            color: white;
            -webkit-user-select: none;
            -moz-user-select: none;
            -ms-user-select: none;
            user-select: none;
            background-attachment: fixed;

            height: 100%;
            min-height: 100vh;
        }

        /* --- HEADER --- */
        header {
            display: flex;
            justify-content: space-between;
            align-items: center;
            width: 100%;
            padding: 0.5rem;
        }

        .logo-container,
        .login-container {
            display: flex;
            align-items: center;
            justify-content: center;
            padding: 8px;
            gap: 6px;
        }

        .logo {
            height: 70px;
            width: auto;
        }

        .section-title {
            color: #007bff;
        }

        .miniTitle {
            width: 100%;
            text-align: center;
            font-size: 1.5rem;
            font-weight: 600;
            color: #38BDF8;
            margin: 0;
            padding: 0;
        }

        .login-container {
            text-decoration: none;
            display: flex;
            align-items: center;
            justify-content: center;
            color: #94A3B8;
            background-color: #ffffff0f;
            border: 1px solid #ffffff1a;
            border-radius: 10px;
            padding: 10px;
            margin: 20px;
            backdrop-filter: blur(10px);
            transition: background-color 0.3s ease;
            cursor: pointer;

            width: auto;
            min-width: 120px;
        }

        .login-container:hover {
            background-color: #ffffff1a;
        }

        .main {
            display: flex;
            flex-direction: column;
            align-items: center;
            padding: 20px;
            gap: 20px;
        }

        .admin-container,
        .dashboard-section {
            display: flex;
            flex-direction: column;
            align-items: center;
            justify-content: center;
            text-align: center;
            margin-bottom: 40px;
        }

        .dashboard-section {
            background: #ffffff1a;
            /* Light background for sections */
            border-radius: 10px;
            padding: 20px;
            margin: 10px 0;
            width: 80%;
            /* Adjusted width for better layout control */
            max-width: 960px;
            /* Max width for larger screens */
            box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);
            /* Subtle shadow for depth */
        }

        .action-btn {
            background-color: #007bff;
            color: #FFFFFF;
            border: none;
            border-radius: 5px;
            padding: 10px 15px;
            cursor: pointer;
            transition: background 0.3s ease;
            align-self: center;
            /* Center button in the section */
            text-decoration: none;
        }

        .action-btn:hover {
            background-color: #0056b3;
        }

        .code-window {
            background-color: #0F172A;
            color: #dcdcdc;
            padding: 15px;
            border-radius: 8px;
            overflow: auto;
            font-size: 0.9rem;
            font-family: 'Courier New', Courier, monospace;
            white-space: pre-wrap;
            text-align: left;
        }

        .scan-info {
            display: flex;
            justify-content: space-between;
            align-items: center;
            width: 90%;
            max-width: 800px;
            margin-top: 20px;
        }

        .scan-status {
            color: #4CAF50;
            font-weight: bold;
        }

        /* --- FOOTER --- */

        footer {
            bottom: 0;
            width: 100%;
            padding: 1rem;
            text-align: center;
            background: transparent;
        }

        footer a {
            color: #E2E8F0;
            /* Light grey color for footer links */
            text-decoration: none;
            font-size: 1rem;
            margin: 0 1rem;
            transition: color 0.3s ease;
        }

        footer a:hover {
            color: #0E99DA;
        }


        /* --- FOR DIFFERENT SIZES OF SCREEN --- */

        @media (max-width: 600px) {
            .scan-info {
                flex-direction: column;
                align-items: flex-start;
            }

            .scan-info span,
            .scan-info button {
                margin-bottom: 10px;
            }
        }

        @media (max-width: 768px) {
            .dashboard-section {
                width: 95%;
                /* More width for smaller screens */
            }

            .logo {
                height: 40px;
                /* Smaller logo for smaller screens */
            }

            .section-title,
            .code-window,
            .action-btn {
                font-size: 0.8rem;
                /* Smaller text for better readability */
            }
        }
    </style>
</head>

<body>

    <header>
        <div class="logo-container">
            <img src="logo_nobg.png" alt="LlamaCloud" class="logo">
        </div>

        <span class="miniTitle">Hello, Admin 👋🏼</span>

        <a onclick="openMainPage()" class="login-container">Log Out
            <i class="fas fa-sign-out-alt"></i>
        </a>
    </header>

    <main>
        <div class="admin-container">
            <!-- Dashboard Sections -->
            <section class="dashboard-section">
                <h2 class="section-title">Check Statistic</h2>
                <a onclick = "openStats()" class="action-btn">Check Statistic</a>
            </section>

            <section class="dashboard-section">
                <h2 class="section-title">Index of AWS</h2>
                <pre class="code-window">
{
    "AWSSearch": {
        "service": "EC2",
        "documentation": "Instance Types",
        "blogPosts": [
            "Scaling Your EC2 Infrastructure",
            "Cost Optimization with EC2"
        ],
        "moreInfo": "Visit AWS official documentation for detailed information."
    }
}
            </pre>
            </section>

            <section class="dashboard-section">
                <h2 class="section-title">Last scan of AWS</h2>
                <div class="scan-info">
                    <span id="lastScanDate">Last Scan of AWS on 20/02/2024</span>
                    <span id="scanStatus" class="scan-status success">Success</span> <!-- Dynamic status indicator -->
                    <button onclick = "scrapeAndSave()" class="action-btn">Scan AWS</button>
                </div>
            </section>
        </div>
    </main>

    <footer>
        <a href="https://youtu.be/dQw4w9WgXcQ" target="_blank">made with 💙 in Braude</a>
        <a href="https://github.com/SecretPasta/Llama_Cloud.git" target="_blank">GitHub</a>
    </footer>

<script>
      function openMainPage() {
    // This will call the Python function 'display_homepage'
    google.colab.kernel.invokeFunction('notebook.display_homepage', [], {})
        .then(function(response) {
            // Handle response here if needed
        })
        .catch(function(error) {
            console.error(error);
        });
}
</script>

<script>
      function openStats() {
    // This will call the Python function 'display_stats'
    google.colab.kernel.invokeFunction('notebook.display_stats', [], {})
        .then(function(response) {
            // Handle response here if needed
        })
        .catch(function(error) {
            console.error(error);
        });
}
</script>

<script>
      function scrapeAndSave() {
    // This will call the Python function 'scrape_and_save'
    google.colab.kernel.invokeFunction('notebook.scrape_and_save', [], {})
        .then(function(response) {
            // Handle response here if needed
        })
        .catch(function(error) {
            console.error(error);
        });
}
</script>

</body>

</html>
"""

# Statistics Page HTML

In [22]:
stats_page = """
<!DOCTYPE html>
<html lang="en">

<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Llama - Statistics</title>

    <!-- For PNG or other formats -->
    <link rel="icon" type="image/png" href="logo_nobg.png">

    <link href="https://fonts.googleapis.com/css2?family=Roboto:wght@400;700&display=swap" rel="stylesheet">
    <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/5.15.3/css/all.min.css">
    <style>
        * {
            box-sizing: border-box;
            margin: 0;
            padding: 0;
        }

        body,
        html {

            -webkit-user-select: none;
            /* Safari */
            -moz-user-select: none;
            /* Firefox */
            -ms-user-select: none;
            /* Internet Explorer/Edge */
            user-select: none;
            /* Non-prefixed version, currently supported by Chrome, Opera and Edge */

            height: 100%;
            font-family: 'Roboto', sans-serif;
            background: linear-gradient(to bottom left, #0F172A, #1A2346, #193251, #142941, #0F172A, #0F172A, #0F172A);
            /* Dark background */
            background-attachment: fixed;
            /* Keep the gradient fixed during scroll */
            color: white;
        }

        /* --- HEADER --- */
        header {
            display: flex;
            justify-content: space-between;
            align-items: center;
            width: 100%;
            padding: 0.5rem;
        }

        .logo-container,
        .login-container {
            display: flex;
            align-items: center;
            justify-content: center;
            padding: 8px;
            gap: 6px;
        }

        .logo {
            height: 70px;
            width: auto;
        }

        .section-title {
            color: #007bff;
        }

        .miniTitle {
            width: 100%;
            text-align: center;
            font-size: 1.5rem;
            font-weight: 600;
            color: #38BDF8;
            margin: 0;
            padding: 0;
        }

        .login-container {
            text-decoration: none;
            display: flex;
            align-items: center;
            justify-content: center;
            color: #94A3B8;
            background-color: #ffffff0f;
            border: 1px solid #ffffff1a;
            border-radius: 10px;
            padding: 10px;
            margin: 20px;
            backdrop-filter: blur(10px);
            transition: background-color 0.3s ease;
            cursor: pointer;

            width: auto;
            min-width: 120px;
        }

        .login-container:hover {
            background-color: #ffffff1a;
        }

        /* --- MAIN --- */

        .main {
            display: flex;
            justify-content: center;
            align-items: center;
            min-height: 100vh;
            /* Use at least 100% of the viewport height */
        }

        .stat-container {
            display: flex;
            flex-direction: column;
            align-items: center;
            justify-content: center;
            text-align: center;
            margin-bottom: 40px;
        }

        .dashboard-section {
            background: #ffffff1a;
            /* Light background for sections */
            border-radius: 10px;
            padding: 20px;
            margin: 20px 0;
            width: 80%;
            /* Adjusted width for better layout control */
            max-width: 960px;
            /* Max width for larger screens */
            box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);
            /* Subtle shadow for depth */
        }


        .section-title {
            text-align: center;
            margin-bottom: 15px;
        }

        /* Line Chart Styling */

        /* --- BAR CHART --- */
        .simple-bar-chart {
            --line-count: 10;
            --line-color: currentcolor;
            --line-opacity: 0.25;
            --item-gap: 2%;
            --item-default-color: #060606;

            height: 10rem;
            display: grid;
            grid-auto-flow: column;
            gap: var(--item-gap);
            align-items: end;
            padding-inline: var(--item-gap);
            --padding-block: 1.5rem;
            /*space for labels*/
            padding-block: var(--padding-block);
            position: relative;
            isolation: isolate;
        }

        .simple-bar-chart::after {
            content: "";
            position: absolute;
            inset: var(--padding-block) 0;
            z-index: -1;
            --line-width: 1px;
            --line-spacing: calc(100% / var(--line-count));
            background-image: repeating-linear-gradient(to top, transparent 0 calc(var(--line-spacing) - var(--line-width)), var(--line-color) 0 var(--line-spacing));
            box-shadow: 0 var(--line-width) 0 var(--line-color);
            opacity: var(--line-opacity);
        }

        .simple-bar-chart>.item {
            height: calc(1% * var(--val));
            background-color: var(--clr, var(--item-default-color));
            position: relative;
            animation: item-height 1s ease forwards
        }

        @keyframes item-height {
            from {
                height: 0
            }
        }

        .simple-bar-chart>.item>* {
            position: absolute;
            text-align: center
        }

        .simple-bar-chart>.item>.label {
            inset: 100% 0 auto 0
        }

        .simple-bar-chart>.item>.value {
            inset: auto 0 100% 0
        }

        /* >--- BAR CHART ---< */

        /* --- PIE CHART --- */

        .pie-chart-container {
            position: relative;
            display: flex;
            justify-content: center;
            align-items: center;
        }

        .pie-chart {
            width: 300px;
            height: 300px;
            border-radius: 50%;
            position: relative;
            background-color: #eee;
            /* Fallback color */
            background-image:
                conic-gradient(from 0deg at 50% 50%,
                    #FFD700 0deg 90deg,
                    /* Chrome */
                    #FF8C00 90deg 162deg,
                    /* Firefox */
                    #FF6347 162deg 207deg,
                    /* Edge */
                    #4682B4 207deg 360deg
                    /* Safari */
                );
        }

        .segment {
            position: absolute;
            width: 100%;
            height: 100%;
            clip-path: circle(50% at 50% 50%);
            display: flex;
            justify-content: center;
            align-items: center;
            text-align: justify;
            font-size: 1;
            /* Hide text */
        }

        .segment::before {
            content: attr(style);
            font-size: 14px;
            position: absolute;
            white-space: nowrap;
            color: transparent;
            /* Hide actual content */
        }

        .labels {
            position: absolute;
            width: 100%;
            height: 100%;
            top: 0;
            left: -30px;
            display: flex;
            justify-content: center;
            align-items: center;
            color: black;
        }

        .label {
            position: absolute;
            /* Adjust the translate value to move labels outward */
            transform: translate(100%, -50%) rotate(var(--angle)) translateX(50%);
            transform-origin: 0% 50%;
            white-space: nowrap;
        }


        /* >--- PIE CHART ---< */



        /* --- FOOTER --- */

        footer {
            bottom: 0;
            width: 100%;
            padding: 1rem;
            text-align: center;
            background: transparent;
        }

        footer a {
            color: #E2E8F0;
            /* Light grey color for footer links */
            text-decoration: none;
            font-size: 1rem;
            margin: 0 1rem;
            transition: color 0.3s ease;
        }

        footer a:hover {
            color: #0E99DA;
        }
    </style>
</head>

<body>

    <header>
        <div class="logo-container">
            <img src="logo_nobg.png" alt="LlamaCloud" class="logo">
        </div>

        <span class="miniTitle">Statistics 📊</span>

        <a onclick= "openAdmin()" class="login-container">Back
            <i class="fas fa-arrow-left"></i>
        </a>
    </header>

    <main>
        <div class="stat-container">

            <!-- Monthly Usage Bar Chart -->
            <div class="dashboard-section">
                <h2 class="section-title">Monthly Usage</h2>
                <div class="simple-bar-chart">
                    <!-- Each "item" represents a month with a unique color and value -->

                    <div class="item" style="--clr: #5EB344; --val: 21"> <!-- Example for January -->
                        <div class="label">Jan</div>
                        <div class="value">21%</div>
                    </div>

                    <div class="item" style="--clr: #FCB72A; --val: 43"> <!-- Example for February -->
                        <div class="label">Feb</div>
                        <div class="value">43%</div>
                    </div>

                    <div class="item" style="--clr: #F8821A; --val: 65"> <!-- Example for March -->
                        <div class="label">Mar</div>
                        <div class="value">65%</div>
                    </div>

                    <div class="item" style="--clr: #E0393E; --val: 53"> <!-- Example for April -->
                        <div class="label">Apr</div>
                        <div class="value">53%</div>
                    </div>
                </div>
            </div>

            <!-- Browser Usage Pie Chart -->
            <div class="dashboard-section">
                <h2 class="section-title">Browser Usage</h2>
                <div class="pie-chart-container">
                    <div class="pie-chart">
                        <div class="segment chrome" data-name="Chrome"></div>
                        <div class="segment firefox" data-name="Firefox"></div>
                        <div class="segment edge" data-name="Edge"></div>
                        <div class="segment safari" data-name="Safari"></div>
                    </div>
                    <div class="labels">
                        <div class="label" style="--angle: 0;">Chrome 25%</div>
                        <div class="label" style="--angle: 90deg;">Firefox 20%</div>
                        <div class="label" style="--angle: 180deg;">Edge 15%</div>
                        <div class="label" style="--angle: 270deg;">Safari 40%</div>
                    </div>
                </div>

            </div>

        </div>
    </main>




    <footer>
        <a href="https://youtu.be/dQw4w9WgXcQ" target="_blank">made with 💙 in Braude</a>
        <a href="https://github.com/SecretPasta/Llama_Cloud.git" target="_blank">GitHub</a>
    </footer>

    <script>
      function openAdmin() {
    // This will call the Python function 'display_admin'
    google.colab.kernel.invokeFunction('notebook.display_admin', [], {})
        .then(function(response) {
            // Handle response here if needed
        })
        .catch(function(error) {
            console.error(error);
        });
}
</script>

</body>

</html>
"""

# Python Code

Methods for swapping screens, invoked from JS code in HTML

In [23]:
def display_homepage():
  clear_output(wait=True)
  display(HTML(landing_page_html))

In [24]:
def display_admin():
  clear_output(wait=True)
  display(HTML(admin_page))

In [25]:
def display_stats():
  clear_output(wait=True)
  display(HTML(stats_page))

In [26]:
def display_results():
    clear_output(wait=True)
    display(HTML(search_results))

Code to have functions called from JavaScript code inside HTML files

In [27]:
output.register_callback('notebook.display_results', display_results)
output.register_callback('notebook.display_homepage', display_homepage)
output.register_callback('notebook.display_admin', display_admin)
output.register_callback('notebook.display_stats', display_stats)
output.register_callback('notebook.scrape_and_save', scrape_and_save)

# Main

In [28]:
display(HTML(landing_page_html))